Por Alex Wicher, estudiante de la Facultad de Matemática, Astronomía, Física y Computación (FAMAF) para el laboratorio de Redes y Sistemas Distribuidos.

# Análisis de enrutamiento, capa de red, con Simplified LSP.

## Resumen.

En este trabajo se presentara un algoritmo de enrutamiento para la capa de red, por lo tanto, implementados en los "nodos intermediarios" como descriptos en el trabajo anterior "Análisis de control de flujo y congestión, capa de transporte" por Alex Wicher. La idea seria maximizar la utilización de la sub-red, entregando al mayor cantidad de paquetes posibles al destino mediante la seleccion de rutas mas efectiva y eficiente posible. Y proveer enrutamiento para cualquier topologia de red.
El algoritmo de enrutamiento propuesto es una implementación simplificada de LSP (Link State Package) Routing, no considera cambios de topologia, pero provee una plataforma decente para extender el algoritmo a una versión completa de LSP.

## Introducción.

En la mayoría de los sistemas operativos de redes existe la capa de red cuya función principal es enrutar los paquetes en una red y entre distintas redes que pueden usar distintas tecnologías. En este trabajo nos vamos a enfocar en el enrutamiento dentro de una sub-red.

El escenario general del laboratorio consiste en una red de topologia de anillo como se ve en la imagen:

<img src="data/lab4.png" width="800" height="400">

Cada nodo posee una capa simplificada de aplicación y una capa de red con diferentes puertas de enlace:

<img src="data/lab4_1.png" width="800" height="400">

El algoritmo de enrutamiento por defecto en este escenario simplemente enviá todos los paquetes por la compuerta de la derecha, causando una obvia mala utilización de la red, y usando caminos mas largos y costosos de lo necesario.

Existen dos casos:
 + Nodo 0 y 2 enviando al nodo 5. Idealmente vamos a tener:

<img src="data/short.png" width="800" height="400">

 + Todos los nodos enviando al nodo 5. Este escenario es muy similar el primero, idealmente las cargas se van a dividir en dos mitades como en la imagen de arriba.

## Métodos.

Simplified LSP usa una matriz de adyacencia para representar la sub-red.
* Simplified LSP consiste de pasos muy similares a los vistos en el teórico, vamos a verlo desde la perspectiva de un nodo individual:
    + Primero se mandan paquetes "hello" por todas las compuertas de salida.
    + Si hay otro nodo conectado este mandara "hello_ack".
    + Al recibirse "hello_ack" se registra el par <ID de nodo vecino, puerta de enlace de salida>.
    + Luego se mandan paquetes "echo" a todos los vecinos registrados.
    + Los nodos vecinos al recibir "echo" inmediatamente los devuelven.
    + Al recibir la devolución del "echo" se calcula el delay y con este:
        + Se guarda el dato en una lista de tipo <ID de nodo vecino, costo>.
        + Se modifica la matriz de adyacencia para considerar el costo.
    + Cuando ya se hallan devuelto todos los "echo", se crea un LSP con la lista completa <ID de nodo vecino, costo> y ID propio del nodo.
    + Se hace Flooding corto con el LSP creado.
    + Cuando se recibe un LSP:
        + Este de guarda en una lista <ID de nodo, LSP>.
        + Se modifica la matriz de adyacencia en base a la lista <ID de nodo vecino, costo> que el LSP provee.
        + Si el LSP es viejo o duplicado, se elimina. Si no, se continua el flooding.
    + Cuando la lista <ID de nodo, LSP> de nodo este llena, se corre Dijkstra, dando como resultado las rutas mas cortas a todos los nodos.
    + Se envia un paquete a la aplicación indicando que el nodo esta listo para enrutar paquetes. La aplicación enviá si tiene paquetes que enviar.
    + Cuando el nodo recibe un paquete, ya sea de otro nodo o la capa de aplicación propia del nodo , se calcula la puerta de enlace de salida en base a los datos dados por Dijstra y enviá el paquete por la misma. La compuerta de salida se registra en una lista <ID destino, compuerta> para ahorra procesamiento.

### Flooding corto

Es un tipo de inundación con registro en base de datos de paquetes en donde aquellos paquetes que llegan y ya figuren en la BD, son eliminados. Este algoritmo fue pensado para reducir el tiempo de simulación en redes grandes, ya que causan muchos eventos. Como consecuencia reduce drásticamente la circulación de paquetes en la sub-red. Si se tuviera que usar en la realidad, la confirmación de los paquetes antes de su registro seria una necesidad. Los nodos mantienen un array del tamaño de la sub-red, indice es el ID del nodo, el valor un LSP. Se mantienen siempre el LSP mas nuevo, los viejos y los duplicados se eliminan para que no se propaguen. También tiene un hop counter igual a la cantidad de nodos de la sub-red.

## Resultados.



## Discusión.
Como mencionado en el resumen el algoritmo es una versión limitada de LSP Routing. No se emiten LSPs excepto en la inicialización de la sub-red, por lo tanto cambios significativos en la subred no son notificados. No se suma/descuenta peso en la matriz de adyacencia por uso/liberación canales de un nodo, ni se notifica esto al resto de los nodos. No se notifican nodos desconectados a la subred.
Se podrían detectar nodos desconectados, enviando paquetes echo cada un cierto tiempo. Se pueden enviar LSPs por flooding "corto" cada cierto tiempo para notificar de cambios de topologia al resto de la sub-red.

###Consideraciones

Por algunas limitaciones de tiempo y mano de obra, el código del algoritmo no esta idealmente organizado ni optimizado. Si se quisiera extender este algoritmo a una versión completa de LSP routing ,o incluso mas, se debería re-formatear el código y analizar las subrutinas básicas que este realiza para tratar de mejorar la base que se provee.
Idealmente se quiere un código que este bien modularizado y se puedan insertar funcionalidades fácilmente como si fueran solo un modulo mas, sin preocuparme por el resto.
Con respecto a la optimización, en Dijkstra se puede remplazar la cola que se usa para marcar los nodos visitados por otros tipos de colas mucho mas eficientes, como es el popular ejemplo de cola con prioridades usado en OSPF.
Con respecto Flooding corto, este se usa en la practica pero desconozco su precisa denominación (1).


## Referencias.

https://famaf.aulavirtual.unc.edu.ar/course/view.php?id=880
https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm
https://www.geeksforgeeks.org/c-program-for-dijkstras-shortest-path-algorithm-greedy-algo-7/
https://www.geeksforgeeks.org/different-types-of-queues-and-its-applications/
https://prepinsta.com/c-program/types-of-queue/
https://stackoverflow.com/questions/10549604/flooding-algorithm-on-a-network (1)
